In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [ ]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2024, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(0, 674)
(0,)
(87880, 674)
(87880,)


In [5]:
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]

In [ ]:
import lightgbm as lgb
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error
predictions = model.predict(X_test_only_numeric)

In [ ]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

3.3054


In [ ]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
import os
load_dotenv() 

mlflow = set_mlflow_tracking()
log_model_to_mlflow(model, X_test_only_numeric, "LGBMRegressor", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:MLflow tracking URI and credentials set.
2025/03/05 10:57:31 INFO mlflow.tracking.fluent: Experiment with name 'LGBMRegressor' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: LGBMRegressor
INFO:src.experiment_utils:Logged mean_absolute_error: 3.3054128762542527
/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://w

🏃 View run peaceful-deer-909 at: https://dagshub.com/ashish159565/spring25_sp.mlflow/#/experiments/4/runs/e49b6b98ec9f444d8586855a89c32dc1
🧪 View experiment at: https://dagshub.com/ashish159565/spring25_sp.mlflow/#/experiments/4
